In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
# Connect to the SQLite database
conn = sqlite3.connect('atwork.sqlite')

In [3]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)

[('artwork',)]


In [4]:
# Read data from a specific table into a pandas DataFrame
table_name = 'artwork'
artwork_data = pd.read_sql_query(f"SELECT * FROM {table_name};", conn)

In [5]:
# Close the connection
conn.close()

In [6]:
artwork_data

,artwork_id,artist_name,title,exhibition_history,is_on_view,date,category_titles,artwork_type_title
0,78503,"Tsugouharu Foujita\nJapanese, 1886–1968",Portrait of Emily Crane Chadbourne,"New York, Reinhardt Galleries, Exhibition of P...",0,1922,Modern Art,Painting
1,53045,"André Lhote\nFrench, 1885-1962",Port of Bordeaux,"The Art Institute of Chicago, Henri Cartier-Br...",0,1914,Modern Art,Painting
2,217464,"Gladys Nilsson\nAmerican, born 1940",The Trogens,None,0,1967,Contemporary Art,Painting
3,93345,"Robert Smithson\nAmerican, 1938–1973",Chalk-Mirror Displacement,"London, Institute of Contemporary Art, “When A...",0,1969,Contemporary Art,Sculpture
4,266051,"Ingrid Pollard\nBritish, born 1953",The Boy Who Watches Ships Go By,None,0,2002,Photography and Media,Photograph
...,...,...,...,...,...,...,...,...
995,61024,"Rockwell Kent\nAmerican, 1882-1971",The Precipice,None,0,n.d.,Prints and Drawings,Print
996,36067,"Rockwell Kent\nAmerican, 1882–1971",Alaska,"Art Institute of Chicago, <a href=""https://www...",0,1919–27,Arts of the Americas,Painting
997,34040,"Rockwell Kent\nAmerican, 1882-1971",Blue Bird,None,0,1919,Prints and Drawings,Print
998,30383,"Rockwell Kent\nAmerican, 1882-1971",Still the Breath Divine does Move/ And the Bre...,None,0,1925,Prints and Drawings,Print


In [7]:
# Create DataFrame
df = pd.DataFrame(artwork_data)

In [8]:
# Split artist_name into name, nationality, and year columns
df[['name', 'nationality_year']] = df['artist_name'].str.split('(', n=1, expand=True)

In [9]:
df[['nationality', 'year']] = df['nationality_year'].str.extract(r'([A-Za-z\s]+),\s(\d{4}–\d{4})')

# Drop unnecessary columns
df.drop(['artist_name', 'nationality_year'], axis=1, inplace=True)

df

,artwork_id,title,exhibition_history,is_on_view,date,category_titles,artwork_type_title,name,nationality,year
0,78503,Portrait of Emily Crane Chadbourne,"New York, Reinhardt Galleries, Exhibition of P...",0,1922,Modern Art,Painting,"Tsugouharu Foujita\nJapanese, 1886–1968",NaN,NaN
1,53045,Port of Bordeaux,"The Art Institute of Chicago, Henri Cartier-Br...",0,1914,Modern Art,Painting,"André Lhote\nFrench, 1885-1962",NaN,NaN
2,217464,The Trogens,None,0,1967,Contemporary Art,Painting,"Gladys Nilsson\nAmerican, born 1940",NaN,NaN
3,93345,Chalk-Mirror Displacement,"London, Institute of Contemporary Art, “When A...",0,1969,Contemporary Art,Sculpture,"Robert Smithson\nAmerican, 1938–1973",NaN,NaN
4,266051,The Boy Who Watches Ships Go By,None,0,2002,Photography and Media,Photograph,"Ingrid Pollard\nBritish, born 1953",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
995,61024,The Precipice,None,0,n.d.,Prints and Drawings,Print,"Rockwell Kent\nAmerican, 1882-1971",NaN,NaN
996,36067,Alaska,"Art Institute of Chicago, <a href=""https://www...",0,1919–27,Arts of the Americas,Painting,"Rockwell Kent\nAmerican, 1882–1971",NaN,NaN
997,34040,Blue Bird,None,0,1919,Prints and Drawings,Print,"Rockwell Kent\nAmerican, 1882-1971",NaN,NaN
998,30383,Still the Breath Divine does Move/ And the Bre...,None,0,1925,Prints and Drawings,Print,"Rockwell Kent\nAmerican, 1882-1971",NaN,NaN


In [10]:
# Create a DataFrame
df = df[['artwork_id', 'name', 'nationality', 'year', 'title', 'exhibition_history', 'is_on_view', 'date', 'category_titles', 'artwork_type_title']].rename(columns={
    'artwork_id': 'ID',
    'exhibition_history': 'exhibition',
    'is_on_view': 'View',
    'artwork_type_title': 'type'
})

# Display DataFrame
df

,ID,name,nationality,year,title,exhibition,View,date,category_titles,type
0,78503,"Tsugouharu Foujita\nJapanese, 1886–1968",NaN,NaN,Portrait of Emily Crane Chadbourne,"New York, Reinhardt Galleries, Exhibition of P...",0,1922,Modern Art,Painting
1,53045,"André Lhote\nFrench, 1885-1962",NaN,NaN,Port of Bordeaux,"The Art Institute of Chicago, Henri Cartier-Br...",0,1914,Modern Art,Painting
2,217464,"Gladys Nilsson\nAmerican, born 1940",NaN,NaN,The Trogens,None,0,1967,Contemporary Art,Painting
3,93345,"Robert Smithson\nAmerican, 1938–1973",NaN,NaN,Chalk-Mirror Displacement,"London, Institute of Contemporary Art, “When A...",0,1969,Contemporary Art,Sculpture
4,266051,"Ingrid Pollard\nBritish, born 1953",NaN,NaN,The Boy Who Watches Ships Go By,None,0,2002,Photography and Media,Photograph
...,...,...,...,...,...,...,...,...,...,...
995,61024,"Rockwell Kent\nAmerican, 1882-1971",NaN,NaN,The Precipice,None,0,n.d.,Prints and Drawings,Print
996,36067,"Rockwell Kent\nAmerican, 1882–1971",NaN,NaN,Alaska,"Art Institute of Chicago, <a href=""https://www...",0,1919–27,Arts of the Americas,Painting
997,34040,"Rockwell Kent\nAmerican, 1882-1971",NaN,NaN,Blue Bird,None,0,1919,Prints and Drawings,Print
998,30383,"Rockwell Kent\nAmerican, 1882-1971",NaN,NaN,Still the Breath Divine does Move/ And the Bre...,None,0,1925,Prints and Drawings,Print


In [11]:
df['exhibition'] = df['exhibition'].apply(lambda x: 0 if 'None' in str(x) else 1)
df

,ID,name,nationality,year,title,exhibition,View,date,category_titles,type
0,78503,"Tsugouharu Foujita\nJapanese, 1886–1968",NaN,NaN,Portrait of Emily Crane Chadbourne,1,0,1922,Modern Art,Painting
1,53045,"André Lhote\nFrench, 1885-1962",NaN,NaN,Port of Bordeaux,1,0,1914,Modern Art,Painting
2,217464,"Gladys Nilsson\nAmerican, born 1940",NaN,NaN,The Trogens,0,0,1967,Contemporary Art,Painting
3,93345,"Robert Smithson\nAmerican, 1938–1973",NaN,NaN,Chalk-Mirror Displacement,1,0,1969,Contemporary Art,Sculpture
4,266051,"Ingrid Pollard\nBritish, born 1953",NaN,NaN,The Boy Who Watches Ships Go By,0,0,2002,Photography and Media,Photograph
...,...,...,...,...,...,...,...,...,...,...
995,61024,"Rockwell Kent\nAmerican, 1882-1971",NaN,NaN,The Precipice,0,0,n.d.,Prints and Drawings,Print
996,36067,"Rockwell Kent\nAmerican, 1882–1971",NaN,NaN,Alaska,1,0,1919–27,Arts of the Americas,Painting
997,34040,"Rockwell Kent\nAmerican, 1882-1971",NaN,NaN,Blue Bird,0,0,1919,Prints and Drawings,Print
998,30383,"Rockwell Kent\nAmerican, 1882-1971",NaN,NaN,Still the Breath Divine does Move/ And the Bre...,0,0,1925,Prints and Drawings,Print


In [12]:
# Replace NaN values in the 'nationality' column with 'Unknown'
df['nationality'].fillna(value='Unknown', inplace=True)
df['year'].fillna(value='Unknown', inplace=True)
df

,ID,name,nationality,year,title,exhibition,View,date,category_titles,type
0,78503,"Tsugouharu Foujita\nJapanese, 1886–1968",Unknown,Unknown,Portrait of Emily Crane Chadbourne,1,0,1922,Modern Art,Painting
1,53045,"André Lhote\nFrench, 1885-1962",Unknown,Unknown,Port of Bordeaux,1,0,1914,Modern Art,Painting
2,217464,"Gladys Nilsson\nAmerican, born 1940",Unknown,Unknown,The Trogens,0,0,1967,Contemporary Art,Painting
3,93345,"Robert Smithson\nAmerican, 1938–1973",Unknown,Unknown,Chalk-Mirror Displacement,1,0,1969,Contemporary Art,Sculpture
4,266051,"Ingrid Pollard\nBritish, born 1953",Unknown,Unknown,The Boy Who Watches Ships Go By,0,0,2002,Photography and Media,Photograph
...,...,...,...,...,...,...,...,...,...,...
995,61024,"Rockwell Kent\nAmerican, 1882-1971",Unknown,Unknown,The Precipice,0,0,n.d.,Prints and Drawings,Print
996,36067,"Rockwell Kent\nAmerican, 1882–1971",Unknown,Unknown,Alaska,1,0,1919–27,Arts of the Americas,Painting
997,34040,"Rockwell Kent\nAmerican, 1882-1971",Unknown,Unknown,Blue Bird,0,0,1919,Prints and Drawings,Print
998,30383,"Rockwell Kent\nAmerican, 1882-1971",Unknown,Unknown,Still the Breath Divine does Move/ And the Bre...,0,0,1925,Prints and Drawings,Print


In [13]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = df.drop(columns=['ID', 'name', 'title', 'year'])
df.head()

,nationality,exhibition,View,date,category_titles,type
0,Unknown,1,0,1922,Modern Art,Painting
1,Unknown,1,0,1914,Modern Art,Painting
2,Unknown,0,0,1967,Contemporary Art,Painting
3,Unknown,1,0,1969,Contemporary Art,Sculpture
4,Unknown,0,0,2002,Photography and Media,Photograph


In [14]:
# Determine the number of unique values in each column.
unique_values = df.nunique()
print(unique_values)

nationality          7
exhibition           2
View                 2
date               479
category_titles     22
type                21
dtype: int64


In [15]:
# Look at date value counts for binning
date_counts = df['date'].value_counts()
print(date_counts)

date
1936, published May 26, 1942    27
n.d.                            22
1978                            18
1970                            17
published 1931                  16
                                ..
1625/50                          1
1877                             1
c. 1881                          1
1882–83                          1
1925                             1
Name: count, Length: 479, dtype: int64


In [16]:
import re
# Drop rows where 'date' is 'n.d.' in the original DataFrame
df.drop(df[df['date'] == 'n.d.'].index, inplace=True)
# Function to handle cases like '1649/1655'
def replace_year_range(text):
    parts = text.split('/')
    if len(parts) > 0:
        return parts[0]
    return None

# Function to extract year from a string
def extract_year(text):
    match = re.search(r'\b\d{4}\b', text)
    if match:
        return match.group()
    return None

# Replace the 'date' values containing '1936, published May 26, 1942' with '1936'
df.loc[df['date'] == '1936, published May 26, 1942', 'date'] = '1936'
df.loc[df['date'] == 'published 1931'] = '1931'
df.loc[df['date'] == '1949, published December 1954'] = '1949'
df.loc[df['date'] == '1882–83'] = '1882'
df.loc[df['date'] == '1880–82'] = '1880'
df['date'] = df['date'].apply(lambda x: replace_year_range(x) if '/' in x else extract_year(x) if isinstance(x, str) else x)

In [17]:
# Check the data types of values in the 'date' column
date_types = df['date'].apply(type).value_counts()
print(date_types)

date
<class 'str'>         951
<class 'NoneType'>     27
Name: count, dtype: int64


In [18]:
df = df[df['date'].notna()]

In [19]:
# Look at date value counts for binning
date_counts = df['date'].value_counts()
print(date_counts)

date
1936    37
1949    29
1931    24
1905    23
1933    23
        ..
1649     1
1662     1
1663     1
1673     1
1580     1
Name: count, Length: 176, dtype: int64


In [20]:
df = df[df['nationality'] != '1931']
df = df[df['nationality'] != '1949']
df = df[df['View'] != '1880']
df = df[df['View'] != '1882']
nat_counts = df['nationality'].value_counts()
print(nat_counts)

nationality
Unknown        790
American       115
French           8
 born Italy      2
Chinese          1
Italian          1
Flemish          1
Name: count, dtype: int64


In [21]:
# Choose a cutoff value and create a list of nationalities to be replaced
# use the variable name `nationalities_to_replace`
cutoff_value_class = 7
nationalities_to_replace = list(nat_counts[nat_counts < cutoff_value_class].index)

# Replace in dataframe
for nat in nationalities_to_replace:
    df['nationality'] = df['nationality'].replace(nat,"Other")

# Check to make sure binning was successful
df['nationality'].value_counts()

nationality
Unknown     790
American    115
French        8
Other         5
Name: count, dtype: int64

In [22]:
# Convert 'date' column to numeric values (years)
df['date'] = pd.to_numeric(df['date'], errors='coerce')

# Drop rows with NaN values in the 'date' column
df.dropna(subset=['date'], inplace=True)

# Convert the 'date' column to integers
df['date'] = df['date'].astype(int)

# Display the updated DataFrame
df

,nationality,exhibition,View,date,category_titles,type
0,Unknown,1,0,1922,Modern Art,Painting
1,Unknown,1,0,1914,Modern Art,Painting
2,Unknown,0,0,1967,Contemporary Art,Painting
3,Unknown,1,0,1969,Contemporary Art,Sculpture
4,Unknown,0,0,2002,Photography and Media,Photograph
...,...,...,...,...,...,...
994,Unknown,0,0,1927,Prints and Drawings,Print
996,Unknown,1,0,1919,Arts of the Americas,Painting
997,Unknown,0,0,1919,Prints and Drawings,Print
998,Unknown,0,0,1925,Prints and Drawings,Print


In [23]:
df['View'] = df['View'].astype(bool)
df['exhibition'] = df['exhibition'].astype(bool)
date_types_1 = df['View'].apply(type).value_counts()
print(date_types_1)

View
<class 'bool'>    915
Name: count, dtype: int64


In [24]:
# Convert categorical data to numeric with `pd.get_dummies`
df_encoded = pd.get_dummies(df)
df_encoded.head()

,exhibition,View,date,nationality_American,nationality_French,nationality_Other,nationality_Unknown,category_titles_AIC Archives,category_titles_African American artists,category_titles_African Diaspora,...,type_Installation,type_Metalwork,type_Mixed Media,type_Painting,type_Photograph,type_Print,type_Religious/Ritual Object,type_Sculpture,type_Textile,type_Vessel
0,True,False,1922,False,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,True,False,1914,False,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,False,False,1967,False,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,True,False,1969,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,False,False,2002,False,False,False,True,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [25]:
# Split our preprocessed data into our features and target arrays
X = df_encoded.drop(columns=['View']).values
y = df_encoded['View'].values

In [26]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [27]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [28]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100


22/22 [==============================] - 1s 3ms/step - loss: 0.4470 - accuracy: 0.8805
Epoch 2/100
22/22 [==============================] - 0s 2ms/step - loss: 0.3243 - accuracy: 0.8819
Epoch 3/100
22/22 [==============================] - 0s 2ms/step - loss: 0.2857 - accuracy: 0.8805
Epoch 4/100
22/22 [==============================] - 0s 2ms/step - loss: 0.2679 - accuracy: 0.8965
Epoch 5/100
22/22 [==============================] - 0s 2ms/step - loss: 0.2590 - accuracy: 0.9009
Epoch 6/100
22/22 [==============================] - 0s 2ms/step - loss: 0.2533 - accuracy: 0.9009
Epoch 7/100
22/22 [==============================] - 0s 2ms/step - loss: 0.2484 - accuracy: 0.9009
Epoch 8/100
22/22 [==============================] - 0s 2ms/step - loss: 0.2465 - accuracy: 0.9009
Epoch 9/100
22/22 [==============================] - 0s 6ms/step - loss: 0.2426 - accuracy: 0.9009
Epoch 10/100
22/22 [==============================] - 0s 3ms/step - loss: 0.2418 - accuracy: 0.9009
Epoch 1

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8/8 - 0s - loss: 0.3172 - accuracy: 0.8996 - 244ms/epoch - 31ms/step
Loss: 0.31724119186401367, Accuracy: 0.8995633125305176


In [32]:
# Initialize an empty DataFrame to store the optimization results
results_df = pd.DataFrame(columns=['Iteration', 'Parameters Changed', 'Accuracy'])

# Define the initial results as a dictionary
initial_results = {'Iteration': ['Initial'],
                   'Parameters Changed': ['N/A'],
                   'Accuracy': [model_accuracy]}  # Assuming model_accuracy is already calculated

# Create DataFrame from the initial results dictionary
results_df = pd.DataFrame(initial_results)

# Print DataFrame to verify
print(results_df)


  Iteration Parameters Changed  Accuracy
0   Initial                N/A  0.899563


In [33]:
# Model Attempt 2
def create_model(input_shape, dropout_rate=0.2, l2_regularizer=0.001):
    model = tf.keras.models.Sequential([
        # Input layer
        tf.keras.layers.Input(shape=input_shape),
        
        # Hidden layers
        tf.keras.layers.Dense(units=128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_regularizer)),
        tf.keras.layers.Dropout(dropout_rate),
        
        tf.keras.layers.Dense(units=64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_regularizer)),
        tf.keras.layers.Dropout(dropout_rate),
        
        # Output layer
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    return model

# Create the model
input_shape = X_train_scaled.shape[1]  # Number of features
model = create_model(input_shape)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test), verbose=1)

model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

changes_made = "Adjusted architecture of model and dropout rate to 0.2 and L2 regularization strength to 0.001"
update_num = len(results_df) + 1
results_df.loc[results_df.shape[0]] = [f'Update #{update_num}', changes_made, model_accuracy]

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               5632      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 13953 (54.50 KB)
Trainable params: 13953 (54.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
22/22 

In [34]:
# Model Attempt 3

# Define the model with an additional layer
def create_model_with_extra_layer(input_shape, dropout_rate=0.3, l2_regularizer=0.0005):
    model = tf.keras.models.Sequential([
        # Input layer
        tf.keras.layers.Input(shape=input_shape),
        
        # Hidden layers
        tf.keras.layers.Dense(units=128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_regularizer)),
        tf.keras.layers.Dropout(dropout_rate),
        
        tf.keras.layers.Dense(units=64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_regularizer)),
        tf.keras.layers.Dropout(dropout_rate),
        
        # Additional hidden layer
        tf.keras.layers.Dense(units=32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_regularizer)),
        tf.keras.layers.Dropout(dropout_rate),
        
        # Output layer
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    return model

# Create the updated model
input_shape = X_train_scaled.shape[1]  # Number of features
updated_model = create_model_with_extra_layer(input_shape)

# Compile the updated model
updated_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train and evaluate the updated model
history = updated_model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test), verbose=1)
model_loss, model_accuracy = updated_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss (Updated): {model_loss}, Accuracy (Updated): {model_accuracy}")

# Record the changes made
changes_made = "Added an additional hidden layer with 32 units"
update_num = len(results_df) + 1
results_df.loc[results_df.shape[0]] = [f'Update #{update_num}', changes_made, model_accuracy]

Epoch 1/100
22/22 [==============================] - 2s 18ms/step - loss: 0.6361 - accuracy: 0.8134 - val_loss: 0.4763 - val_accuracy: 0.8821
Epoch 2/100
22/22 [==============================] - 0s 6ms/step - loss: 0.4652 - accuracy: 0.8805 - val_loss: 0.4056 - val_accuracy: 0.8821
Epoch 3/100
22/22 [==============================] - 0s 6ms/step - loss: 0.4464 - accuracy: 0.8848 - val_loss: 0.3854 - val_accuracy: 0.8821
Epoch 4/100
22/22 [==============================] - 0s 5ms/step - loss: 0.4153 - accuracy: 0.8834 - val_loss: 0.3752 - val_accuracy: 0.8996
Epoch 5/100
22/22 [==============================] - 0s 8ms/step - loss: 0.3972 - accuracy: 0.8790 - val_loss: 0.3692 - val_accuracy: 0.9039
Epoch 6/100
22/22 [==============================] - 0s 5ms/step - loss: 0.3852 - accuracy: 0.8892 - val_loss: 0.3622 - val_accuracy: 0.8996
Epoch 7/100
22/22 [==============================] - 0s 10ms/step - loss: 0.3629 - accuracy: 0.8878 - val_loss: 0.3617 - val_accuracy: 0.8996
Epoch 8/100

In [35]:
# Save the results to a CSV file
results_df.to_csv('model_optimization_results.csv', index=False)